# Portfolio Management

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yfinance as yf

plt.style.use("ggplot")

## Portfolio Formation

We selected 10 representative U.S. tech and growth stocks for the period 2020-01-01 to 2024-12-31, and retrieved the daily stock data using the yfinance package:

In [ ]:
tickers = ["NVDA", "AMD", "TSM", "AVGO", "MSFT", "META", "PLTR", "CVX", "COP", "NEM"]
groups= []

for ticker in tickers:
    data = yf.download(ticker, start="2020-01-01", end="2024-12-31")
    data.columns = data.columns.droplevel(1)  # Remove multi-level column index
    data.insert(0, "Ticker", ticker)  # Add ticker column
    data.reset_index(inplace=True)
    groups.append(data)

multidata = pd.concat(groups)
multidata
# Save to current working directory as 'multidata.csv'
multidata.to_csv("multidata.csv", index=False)

In [12]:
multidata

Price,Date,Ticker,Close,High,Low,Open,Volume
0,2020-01-02,NVDA,5.971746,5.971746,5.892342,5.942872,237536000
1,2020-01-03,NVDA,5.876163,5.919972,5.827126,5.852018,205384000
2,2020-01-06,NVDA,5.900806,5.906033,5.756684,5.782820,262636000
3,2020-01-07,NVDA,5.972245,6.018045,5.884128,5.929182,314856000
4,2020-01-08,NVDA,5.983446,6.024766,5.927938,5.968013,277108000
...,...,...,...,...,...,...,...
1252,2024-12-23,NEM,37.758018,37.856964,37.065392,37.609597,8591600
1253,2024-12-24,NEM,37.906441,37.916334,37.471073,37.797599,3549300
1254,2024-12-26,NEM,37.688755,38.203278,37.688755,37.906439,5896400
1255,2024-12-27,NEM,37.441387,37.817385,37.114862,37.213810,5456100


### Data Cleaning Verification

In [19]:
print("Dataset Structure:")
multidata.info()

Dataset Structure:
<class 'pandas.core.frame.DataFrame'>
Index: 12382 entries, 0 to 1256
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    12382 non-null  datetime64[ns]
 1   Ticker  12382 non-null  object        
 2   Close   12382 non-null  float64       
 3   High    12382 non-null  float64       
 4   Low     12382 non-null  float64       
 5   Open    12382 non-null  float64       
 6   Volume  12382 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 773.9+ KB


## Annualising Returns and Volatility

To provide a broader picture of our portfolio's performance, let's calculate the annualised portfolio return and volatility. These are important metrics for evaluating and discussing portfolios. They are also used frequently in more advanced financial analyses.

In [ ]:
TRADING_DAYS = 252
annualised_return = p_returns.mean() * TRADING_DAYS
annualised_volatility = p_volatility * np.sqrt(TRADING_DAYS)

print(
    f"Annualised return is: {annualised_return}, Annualised volatility is : {annualised_volatility}"
    
)

## Sharpe Ratio

The Sharpe Ratio is a measure that helps investors understand the risk-adjusted return of an investment. A high Sharpe ratio indicates that the portfolio's returns are higher for each unit of risk taken on. In contrast, a lower Sharpe Ratio indicates a less favorable risk-reward trade-off, with the potential for lower returns relative to the amount of risk being assumed.

The Sharpe Ratio is calculated using annualised portfolio returns, portfolio volatility (as the measure of risk), and the *risk-free rate*. The risk-free rate is often derived from the yield of a theoretically risk-free investment, typically a government bond. The Sharpe Ratio is the average return earned in excess of the risk-free rate per unit of volatility or total risk.

### Exercise: Looking Sharpe

Calculate the Sharpe Ratio for our portfolio by applying the formula below.

$$ \text{Sharpe Ratio} = \frac{R_p - R_f}{\sigma_p} $$

- $R_p$ is our annualised portfolio return
- $R_f$ is the risk-free rate
- $\sigma_p$ is our annualised volatility


In [ ]:
RFR = 0.0438
sharpe = (annualised_return - RFR) / annualised_volatility
print(f"Sharpe Ratio is: {sharpe}")